In [4]:
# keep_only_allowed_56_soomin.py  (노트북 셀에 그대로 실행)
from pathlib import Path
import pandas as pd
import unicodedata as ud

# 1) 파일명(품목군)별 ✅ "사용 가능"만 남기는 화이트리스트
KEEP = {
    "감자": {"수미"},
    "고구마": {"밤고구마", "호박고구마"},
    "단호박": {"단호박"},
    "적양배추": {"적채"},
    # 폴더에 '토마토' 또는 '토마토전체'로 있을 수 있어 둘 다 허용
    "토마토": {"대추방울", "방울토마토", "완숙토마토", "토마토(일반)"},
    "토마토전체": {"대추방울", "방울토마토", "완숙토마토", "토마토(일반)"},
}

# 파일명 별칭(동일 취급)
ALIASES = {
    "감자": {"감자"},
    "고구마": {"고구마"},
    "단호박": {"단호박"},
    "적양배추": {"적양배추", "적 양배추"},
    "토마토": {"토마토"},
    "토마토전체": {"토마토전체", "토마토 전체", "토마토"},  # '토마토'가 들어오면 우선 '토마토전체'로 매핑
}

# 세부품목 컬럼 후보
COL_CANDS = ["품목명", "세부 품목", "세부품목", "품목", "세부품목명", "세부 품목명"]

def norm(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ud.normalize("NFC", s).strip()

# 2) 작업 폴더 자동 결정 (현재/상하위/고정경로/트리검색)
def pick_root():
    cwd = Path.cwd().resolve()
    cands = [
        cwd, cwd / "56 수민", cwd.parent / "56 수민",
        Path.home() / "Documents/GitHub/GDF_Final_G3/sojin/56 수민",
    ]
    for p in cands:
        if p.exists() and p.is_dir() and list(p.glob("*.csv")):
            return p.resolve()
    hits = [h for h in cwd.glob("**/56 수민") if list(h.glob("*.csv"))]
    if hits:
        return hits[0].resolve()
    raise SystemExit(f"[에러] '56 수민' 폴더를 찾지 못했습니다. CWD={cwd}")

root = pick_root()
print(f"[INFO] 작업 폴더: {root}")

# 3) 파일명 → 규칙키 매핑
def to_key(stem: str):
    s = norm(stem)
    # '토마토전체' 우선 매핑
    if "토마토전체" in s or ("토마토" in s and "전체" in s):
        return "토마토전체"
    for key, names in ALIASES.items():
        for n in names:
            if s == norm(n) or s.startswith(norm(n)):
                return key
    return None

# 4) CSV 수집 (이미 *_cleaned.csv 제외)
files = [p for p in sorted(root.glob("*.csv")) if not p.name.endswith("_cleaned.csv")]
print("[INFO] 발견 CSV:", [p.stem for p in files])
if not files:
    raise SystemExit("[중지] 처리할 CSV가 없습니다.")

total_in = total_out = 0
processed = 0

for fp in files:
    key = to_key(fp.stem)
    if key not in KEEP:
        print(f"[SKIP-NORULE] {fp.name}: 이번 화이트리스트(KEEP)에 없는 파일 → 건너뜀")
        continue

    keep_set = {norm(x) for x in KEEP[key]}

    # 5) 읽기 (인코딩 자동)
    try:
        df = pd.read_csv(fp, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="cp949")

    # 6) 정규화/공백 정리
    df.columns = [norm(c) for c in df.columns]
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).map(norm)

    # 7) 세부품목 컬럼 탐지
    col = next((c for c in COL_CANDS if c in df.columns), None)
    out_dir = fp.parent / "cleaned"
    out_dir.mkdir(exist_ok=True, parents=True)
    out_path = out_dir / f"{fp.stem}_cleaned.csv"

    if col is None:
        # 컬럼 못 찾으면 원본 그대로 저장(추적 용이)
        df.to_csv(out_path, index=False, encoding="utf-8-sig")
        print(f"[SKIP-NOCOL] {fp.name}: 세부품목 컬럼 미발견 → 원본 그대로 저장: {out_path.name}")
        total_in += len(df); total_out += len(df); processed += 1
        continue

    before = len(df)
    # ✅ 화이트리스트만 남김 (표에 없는 모든 값은 전부 제거)
    df2 = df[df[col].isin(keep_set)].copy()
    after = len(df2)

    df2.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"[OK] {fp.name}: {before} → {after} (보존 {after}행) | 키='{key}' | 컬럼='{col}' "
          f"| 유지 세트: {sorted(keep_set)} | 저장: {out_path.name}")

    total_in += before; total_out += after; processed += 1

print(f"\n[요약] 처리된 파일: {processed}개 / 발견: {len(files)}개 | 총 {total_in}행 → {total_out}행")
print(f"[완료] 결과는 '{root}/cleaned' 폴더에서 확인하세요.")


[INFO] 작업 폴더: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/56 수민
[INFO] 발견 CSV: ['감자', '고구마', '단호박', '적양배추', '토마토전체']
[OK] 감자.csv: 40194 → 3654 (보존 3654행) | 키='감자' | 컬럼='품목명' | 유지 세트: ['수미'] | 저장: 감자_cleaned.csv
[OK] 고구마.csv: 25578 → 7308 (보존 7308행) | 키='고구마' | 컬럼='품목명' | 유지 세트: ['밤고구마', '호박고구마'] | 저장: 고구마_cleaned.csv
[OK] 단호박.csv: 7308 → 3654 (보존 3654행) | 키='단호박' | 컬럼='품목명' | 유지 세트: ['단호박'] | 저장: 단호박_cleaned.csv
[OK] 적양배추.csv: 7308 → 0 (보존 0행) | 키='적양배추' | 컬럼='품목명' | 유지 세트: ['적채'] | 저장: 적양배추_cleaned.csv
[OK] 토마토전체.csv: 32886 → 14616 (보존 14616행) | 키='토마토전체' | 컬럼='품목명' | 유지 세트: ['대추방울', '방울토마토', '완숙토마토', '토마토(일반)'] | 저장: 토마토전체_cleaned.csv

[요약] 처리된 파일: 5개 / 발견: 5개 | 총 113274행 → 29232행
[완료] 결과는 '/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/56 수민/cleaned' 폴더에서 확인하세요.
